# Deep Network for Design Parameters for FWUAV

Importing the Modules

In [1]:
# Modules to import for data preprocessing
import numpy as np
import pandas as pd
import random as rnd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Modules to import for Torch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Check for GPU device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Loading the Data

In [3]:
# Loading the data into the pandas dataframe

# Printing the datafram 
print("\n Data loaded into pandas : \n", data)

# Reading the Data
data = pd.read_csv("Design_Data.csv", header=None)

# Set the first column to object type
data[:0] = data[:0].astype('object')

# Set dtype for the rest of the columns to float32
data.iloc[:, 1:] = data.iloc[:, 1:].astype('float32')

print("\n---------------------------------------------------------------------------\n")

# Inserting the data columns 
data.columns = [
    'Airfoil','Wing Span', 'Taper Ratio', 'Aspect Ratio', 'Flapping Period',
    'Airspeed', 'Angle of Attack', 'Lift', 'Induced Drag'
]

print("\n---------------------------------------------------------------------------\n")

# Split the data into features and targets
X = data[[
    'Airfoil','Wing Span', 'Taper Ratio', 'Aspect Ratio', 'Flapping Period',
    'Airspeed', 'Angle of Attack'
]]
print("Features : \n",X)

print("\n---------------------------------------------------------------------------\n")

print("Targets : \n",X)
y = data[['Lift', 'Induced Drag']]



 Data loaded into pandas : 
 <module 'torch.utils.data' from '/home/stimp/anaconda3/envs/mlenv/lib/python3.8/site-packages/torch/utils/data/__init__.py'>

---------------------------------------------------------------------------


---------------------------------------------------------------------------

Features : 
         Airfoil Wing Span Taper Ratio Aspect Ratio Flapping Period Airspeed  \
0      naca8304       0.4         1.5          0.2             0.4      3.0   
1      naca8304       0.4         1.5          0.2             0.4      3.0   
2      naca8304       0.4         1.5          0.2             0.4      3.0   
3      naca8304       0.4         1.5          0.2             0.4      3.0   
4      naca8304       0.4         1.5          0.2             0.4      4.0   
...         ...       ...         ...          ...             ...      ...   
10682  naca2412       0.8         2.0          0.4             1.1      6.5   
10683  naca0012       0.8         2.0       

# Splitting the data into training and testing 

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the split
print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)


Training features shape: (8549, 7)
Testing features shape: (2138, 7)
Training target shape: (8549, 2)
Testing target shape: (2138, 2)


In [8]:
print(X_train.values)

[['naca0012' 0.3499999940395355 3.0 ... 0.75 3.0 20.0]
 ['naca2412' 0.800000011920929 2.0 ... 0.20000000298023224 2.0 20.0]
 ['goe225' 0.800000011920929 2.0 ... 0.5 4.5 15.0]
 ...
 ['naca0012' 1.2000000476837158 2.0 ... 0.20000000298023224 5.0 20.0]
 ['naca8304' 0.4000000059604645 2.5 ... 1.0 3.0 15.0]
 ['naca2412' 0.30000001192092896 3.0 ... 0.75 4.0 20.0]]


# Defining the Model

In [9]:
class DeepNN(nn.Module):
    def __init__(self):
        super(DeepNN, self).__init__()
        self.fc1 = nn.Linear(7, 128)  # Change input features to 11
        self.dropout1 = nn.Dropout(0.5)  # Dropout layer
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(0.5)  # Dropout layer
        self.fc3 = nn.Linear(64, 32)
        self.dropout3 = nn.Dropout(0.5)  # Dropout layer
        self.fc4 = nn.Linear(32, 2)    # Change output features to 3
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.fc4(x)
        return x


In [10]:
# Convert to 2D PyTorch tensors
# X_train = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.values, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test.values, dtype=torch.float32).to(device)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [6]:
print(y_test)

NameError: name 'y_test' is not defined

# Training and eval of the model 

In [8]:
# Move the model to GPU if available
model = DeepNN().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early stopping
patience = 100
best_loss = float('inf')
best_model_state = None
no_improvement_epochs = 0

# Training the model
num_epochs = 100
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    train_loss = criterion(outputs, y_train)
    train_loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        test_loss = criterion(test_outputs, y_test)

    train_losses.append(train_loss.item())
    test_losses.append(test_loss.item())

    if test_loss < best_loss:
        best_loss = test_loss
        best_model_state = model.state_dict()
        no_improvement_epochs = 0
    else:
        no_improvement_epochs += 1

    if no_improvement_epochs >= patience:
        print(f'Early stopping at epoch {epoch+1}')
        break

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item()}, Test Loss: {test_loss.item()}')

# Load the best model state
if best_model_state is not None:
    model.load_state_dict(best_model_state)


Epoch [10/100], Train Loss: 41.23115921020508, Test Loss: 7620.916015625
Epoch [20/100], Train Loss: 35.96488952636719, Test Loss: 7614.48388671875
Epoch [30/100], Train Loss: 35.16935348510742, Test Loss: 7613.89404296875
Epoch [40/100], Train Loss: 34.270484924316406, Test Loss: 7614.48828125
Epoch [50/100], Train Loss: 33.50812530517578, Test Loss: 7613.828125
Epoch [60/100], Train Loss: 33.00544738769531, Test Loss: 7613.873046875
Epoch [70/100], Train Loss: 32.55061340332031, Test Loss: 7613.79150390625
Epoch [80/100], Train Loss: 32.25320053100586, Test Loss: 7613.7392578125
Epoch [90/100], Train Loss: 31.953275680541992, Test Loss: 7613.6376953125
Epoch [100/100], Train Loss: 31.509950637817383, Test Loss: 7613.44384765625


# testig the data from ptera software


In [9]:
import numpy as np 
import pandas as pd
from Mark4 import simulation, Mk4SaveDataToCSV
from SelfFunctions import extract_second_cycle

In [10]:
import random

def generate_random_decimal(start, end):
    return random.uniform(start, end)

# Example usage
start_value = 1.5
end_value = 5.5
random_number = generate_random_decimal(start_value, end_value)
print("Random number:", random_number)


Random number: 4.387348340381166


In [11]:
Air_Speed = generate_random_decimal(2, 6)
Angles_of_Attack = generate_random_decimal(-35, 35)
FlappingPeriod = generate_random_decimal(0.5, 1)
Normalisedtime =generate_random_decimal(0,1)
mw_root_chord = generate_random_decimal(0.3, 0.1)
mw_wingspan = generate_random_decimal(0.5, 2)
mw_tip_chord = generate_random_decimal(0.1,0.5)
tail_bposition = generate_random_decimal(0.45, 1.2)
tail_root_chord = generate_random_decimal(0.3,0.7)
tail_tip_chord = generate_random_decimal(0.001,0.1)
tail_wingspan = generate_random_decimal(0.5, 1)

print(f"Air Speed: {Air_Speed}\n"
      f"Angles of Attack: {Angles_of_Attack}\n"
      f"Flapping Period: {FlappingPeriod}\n"
      f"Main Wing Root Chord: {mw_root_chord}\n"
      f"Main Wing Wingspan: {mw_wingspan}\n"
      f"Main Wing Tip Chord: {mw_tip_chord}\n"
      f"Tail Base Position: {tail_bposition}\n"
      f"Tail Root Chord: {tail_root_chord}\n"
      f"Tail Tip Chord: {tail_tip_chord}\n"
      f"Tail Wingspan: {tail_wingspan}")


Air Speed: 4.17965874113957
Angles of Attack: -0.0836540286168983
Flapping Period: 0.9904541624018384
Main Wing Root Chord: 0.2301170972790676
Main Wing Wingspan: 1.3269527417088174
Main Wing Tip Chord: 0.3659938259967198
Tail Base Position: 0.8394697809259538
Tail Root Chord: 0.39156864384421824
Tail Tip Chord: 0.014636378344145058
Tail Wingspan: 0.9416947383745498


In [12]:
lift, pitchingmoment, inducedsideforces = simulation(va= Air_Speed, aoa = Angles_of_Attack, fp = FlappingPeriod, mw_airfoil = "naca8304", mw_root_chord = mw_root_chord, mw_wingspan = mw_wingspan, mw_tip_chord = mw_tip_chord, tail_bposition = tail_bposition, tail_type = "V-Tail", tail_root_chord = tail_root_chord , tail_tip_chord = tail_tip_chord, tail_wingspan = tail_wingspan, tail_airfoil = "naca0004" )
print(lift, pitchingmoment, inducedsideforces)

# Flatten the arrays
lift = lift.flatten()
induced_drag = pitchingmoment.flatten()
pitching_moment = inducedsideforces.flatten()
print("Before Second Cycle Extraction :  ", lift.shape, induced_drag.shape, pitching_moment.shape)
print("Before Second Cycle Extraction value:  ", lift, induced_drag, pitching_moment)

# Extract the second cycle from the data
lift, induced_drag, pitching_moment = extract_second_cycle(lift, induced_drag, pitching_moment)
print("After Second Cycle Extraction :  ",lift.shape, induced_drag.shape, pitching_moment.shape)
print("After Second Cycle Extraction value:  ", lift, induced_drag, pitching_moment)

file_name = "test.csv"
Mk4SaveDataToCSV(FlappingPeriod, Air_Speed, Angles_of_Attack, lift, pitching_moment, induced_drag, file_name, 
     mw_root_chord, mw_wingspan, mw_tip_chord, tail_bposition, tail_root_chord, tail_tip_chord, tail_wingspan)

Simulating:100% |█████████████████████████████████████████████████| Elapsed: 01:05, Remaining: 00:00


[[-6.26758308e-02 -1.01021762e+01 -6.35367133e-02  5.34215707e-01
   9.02428224e-01  1.14539195e+00  1.30809939e+00  1.41375811e+00
   1.47601916e+00  1.50388223e+00  1.50454810e+00  1.48841312e+00
   1.52396784e+00  1.22444872e+00  5.50627760e-01  7.39731560e-01
   8.03854125e-01  1.38176798e+00  1.48304310e+00  5.95527306e-02
   1.97305083e+00  1.25095817e+00  1.14015566e+00  5.69421472e-03
  -5.38971261e-01 -9.72258127e-01 -1.36386060e+00 -1.73501195e+00
  -2.09560067e+00 -2.44877740e+00 -2.79489301e+00 -3.13577425e+00
  -3.46435126e+00 -3.78483465e+00 -4.08845263e+00 -4.37538142e+00
  -4.64077511e+00 -4.88023011e+00 -5.09267653e+00 -5.27441582e+00
  -5.42417544e+00 -5.54043476e+00 -5.62284340e+00 -5.67131424e+00
  -5.68706921e+00 -5.67116873e+00 -5.62533959e+00 -5.55143734e+00
  -5.45130145e+00 -5.32634796e+00 -5.17696229e+00 -5.00110208e+00
  -4.79361904e+00 -4.55528030e+00 -4.30941764e+00 -4.21266473e+00
  -4.02696082e+00 -3.67863697e+00 -3.41805614e+00 -3.09219456e+00
  -2.93033

In [14]:


# Input array
input_array = [Air_Speed, Angles_of_Attack, FlappingPeriod, 0.023809523809523808,0,0,0, mw_root_chord, mw_wingspan, mw_tip_chord, tail_bposition, tail_root_chord, tail_tip_chord, tail_wingspan]
input_array = np.array(input_array)
input_array = input_array.reshape(1, -1)

indices_to_remove = [4, 5, 6]
normalised_input_array = np.delete(input_array, indices_to_remove, axis=1)

# Convert to PyTorch tensor
sample_input = torch.tensor(normalised_input_array, dtype=torch.float32)

# Flatten the input tensor to match model's expected input shape
sample_input = sample_input.view(1, -1)

# Send the tensor to the device (CPU or GPU)
sample_input = sample_input.to(device)

# Print the tensor
print("Sample Input Tensor:", sample_input)

# Make predictions
with torch.no_grad():
    predictions = model(sample_input)
    print(predictions)

predictions_numpy = predictions.cpu().numpy()
print(predictions_numpy.reshape(1, -1))
full_prediction_numpy = [Air_Speed, Angles_of_Attack, FlappingPeriod, 0.023809523809523808,predictions_numpy[0][0],predictions_numpy[0][1],predictions_numpy[0][2], mw_root_chord, mw_wingspan, mw_tip_chord, tail_bposition, tail_root_chord, tail_tip_chord, tail_wingspan]
print(full_prediction_numpy)
print(predictions_numpy[0][0],predictions_numpy[0][1],predictions_numpy[0][2])


Sample Input Tensor: tensor([[ 4.1797, -0.0837,  0.9905,  0.0238,  0.2301,  1.3270,  0.3660,  0.8395,
          0.3916,  0.0146,  0.9417]], device='cuda:0')
tensor([[-0.5657, -0.0430,  0.1406]], device='cuda:0')
[[-0.5656901  -0.04296688  0.14059857]]
[4.17965874113957, -0.0836540286168983, 0.9904541624018384, 0.023809523809523808, -0.5656901, -0.04296688, 0.14059857, 0.2301170972790676, 1.3269527417088174, 0.3659938259967198, 0.8394697809259538, 0.39156864384421824, 0.014636378344145058, 0.9416947383745498]
-0.5656901 -0.04296688 0.14059857
